In [10]:
import networkx as nx

In [11]:
#['Abhay', 'Vijay', 'Ragha', 'Balaji', 'Anu'] ['Abhay', 'Ragha', 'Aryan', 'Abhishek', 'Adya']
social_g = {
  'Aryan': [('Adya', 4), ('Vijay', 2), ('Ragha', 1)],
  'Adya': [('Aryan', 1), ('Vijay', 2), ('Abhishek', 3)], 
  'Abhay': [('Aryan', 1), ('Abhishek', 4), ('Anu', 5)],
  'Abhishek': [('Adya', 2), ('Vijay', 3)],
  'Akash': [('Balaji',1), ('Ragha', 5)],
  'Vijay': [('Abhay', 1), ('Adya', 4), ('Abhishek', 5)],
  'Ragha': [('Aryan', 1), ('Abhay', 4), ('Vijay', 5)],
  'Anu': [('Balaji', 1), ('Aryan', 4)],
  'Balaji': [('Adya', 1), ('Aryan', 4), ('Abhay', 5)],
  'Nigga': [],
}
Social = nx.DiGraph()
for node in social_g:
    for k in social_g[node]:
        Social.add_edge(node, k[0], weight= k[1])

In [12]:
def generate_user_follow_suggestions(user_follow_data, user):
    # Create a graph to represent user follow relationships
    Social = nx.Graph()

    # Add user nodes and their follow relationships as edges
    for node in user_follow_data:
        for k in user_follow_data[node]:
            Social.add_edge(node, k[0], weight= k[1])

    # Perform user follow suggestions using graph algorithms
    suggestions = []
    degree_centrality = nx.degree_centrality(Social)
    eigenvector_centrality = nx.eigenvector_centrality(Social, max_iter=1000)
    sorted_users = sorted(eigenvector_centrality.items(), key=lambda x: x[1], reverse=True)
    user_to_recommend = user
    print(degree_centrality)

    
    if(user not in degree_centrality):
        suggestions += list(nx.articulation_points(Social))
        for user, centrality in sorted_users:
            if user != user_to_recommend and Social.has_edge(user_to_recommend, user) == False:
                suggestions.append(user)
                if len(suggestions) >= 5:  # Limit the number of recommendations
                    break

    else:
        pagerank_scores = nx.pagerank(Social)
        neighbours = list(Social.neighbors(user))
        n = 5
        neighbours_sorted = sorted(neighbours,
            key= lambda x: pagerank_scores[x], reverse=True)
    
        suggestions += neighbours_sorted[:n]

    return suggestions

# Example user follow data (follower, following)
user_follow_data = social_g

user_follow_suggestions = generate_user_follow_suggestions(user_follow_data, 'Vijay')
print(user_follow_suggestions)

{'Aryan': 0.75, 'Adya': 0.5, 'Vijay': 0.625, 'Ragha': 0.5, 'Abhishek': 0.375, 'Abhay': 0.75, 'Anu': 0.375, 'Akash': 0.25, 'Balaji': 0.625}
['Abhay', 'Ragha', 'Aryan', 'Abhishek', 'Adya']
